In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/next-utterance-prediction-nlp-2025/val/cache-ed401026e9103c65.arrow
/kaggle/input/next-utterance-prediction-nlp-2025/val/state.json
/kaggle/input/next-utterance-prediction-nlp-2025/val/dataset_info.json
/kaggle/input/next-utterance-prediction-nlp-2025/val/data-00000-of-00001.arrow
/kaggle/input/next-utterance-prediction-nlp-2025/val/cache-cc3928cf36c83499.arrow
/kaggle/input/next-utterance-prediction-nlp-2025/val/cache-00884308fd07a1b2.arrow
/kaggle/input/next-utterance-prediction-nlp-2025/test/state.json
/kaggle/input/next-utterance-prediction-nlp-2025/test/dataset_info.json
/kaggle/input/next-utterance-prediction-nlp-2025/test/data-00000-of-00001.arrow
/kaggle/input/next-utterance-prediction-nlp-2025/train/state.json
/kaggle/input/next-utterance-prediction-nlp-2025/train/dataset_info.json
/kaggle/input/next-utterance-prediction-nlp-2025/train/cache-e2d63a970d5d5c31.arrow
/kaggle/input/next-utterance-prediction-nlp-2025/train/data-00000-of-00001.arrow
/kaggle/input/next-

In [7]:
from datasets import Dataset 
from datasets import load_from_disk

# Load from arrow file directly
train = Dataset.from_file("/kaggle/input/endsem-eval-nlp/val.arrow")
test = Dataset.from_file("/kaggle/input/endsem-eval-nlp/test.arrow")
val = Dataset.from_file("/kaggle/input/endsem-eval-nlp/train.arrow")

In [8]:
print(train)
print(val)
print(test)
print(type(train))

Dataset({
    features: ['input_text', 'target_text'],
    num_rows: 4008
})
Dataset({
    features: ['input_text', 'target_text'],
    num_rows: 576
})
Dataset({
    features: ['input_text', 'target_text'],
    num_rows: 968
})
<class 'datasets.arrow_dataset.Dataset'>


In [9]:
train_df = train.to_pandas()
test_df= test.to_pandas()
val_df = val.to_pandas()

In [10]:
train_df.head(4)

,input_text,target_text
0,T: Hi you how to do it today? [SEP] P: Great. ...,I'm doing well. Thanks for asking.
1,T: Hi you how to do it today? [SEP] P: Great. ...,So you're doing great.
2,T: Hi you how to do it today? [SEP] P: Great. ...,I know your brother brought you in today and h...
3,T: Hi you how to do it today? [SEP] P: Great. ...,"Alright, so you feel like, everything's kind o..."


In [11]:
train_df.iloc[0, 0]

'T: Hi you how to do it today? [SEP] P: Great. How are you?'

In [12]:
train_df.iloc[0 ,  0]

'T: Hi you how to do it today? [SEP] P: Great. How are you?'

In [13]:
train_df.iloc[3,  0]

"T: Hi you how to do it today? [SEP] P: Great. How are you? [SEP] T: I'm doing well. Thanks for asking. [SEP] T: So you're doing great. [SEP] P: I'm doing awesome. [SEP] T: I know your brother brought you in today and he had expressed some concerns about your mood. Do you know what that's about? [SEP] P: I, I think he's worrying for no reason. Because I, you know, I for the last like week and a half, I have been following my passion, which is baking. And I didn't realize this before, because I worked in an office. And I just realized that that's not what I want to do with my life. And I know what I want to do with my life now, and I've never known before. So this is an amazing feeling. And what I want to do is I want to start my own baking business. And so what happened was That a couple. Two weeks ago, my nephew had a bake sale. And I've always I've always been like baking, but I've always been like from the box or like from, you know, whatever. So I tried for the first time making br

In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")
tokenizer.pad_token = tokenizer.eos_token


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

2025-04-14 23:43:15.824156: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744674196.017810      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744674196.074286      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [15]:
def preprocess(example):
    full_text = example['input_text'] + tokenizer.eos_token + example['target_text'] + tokenizer.eos_token
    tokenized = tokenizer(full_text, truncation=True, padding='max_length', max_length=512)
    return {
        'input_ids': tokenized['input_ids'],
        'attention_mask': tokenized['attention_mask'],
        'labels': tokenized['input_ids']  # For language modeling
    }

### We are adding both `input_ids` and `labels` in the output of the preprocessing function, even though they have the same values (i.e., ``tokenized['input_ids'])``. This is because we are doing a next-token prediction task (language modeling). 

### In preprocessing, we are combining the input (like a prompt or training part) and the target (like the expected output or test part) into a single sequence. Then, we tokenize this combined sequence. Both input_ids and labels are set to this tokenized sequence. Later during training, the model will use input_ids as input and labels to calculate the loss by predicting the next tokens in the sequence.

In [16]:
processed = [preprocess(row) for _, row in train_df.iterrows()]
val_processed = [preprocess(row) for _, row in val_df.iterrows()]
test_processed = [preprocess(row) for _, row in test_df.iterrows()]
tokenized_dataset = Dataset.from_list(processed)
tokenized_val_dataset =  Dataset.from_list(val_processed)
tokenized_test_dataset = Dataset.from_list(test_processed)

In [17]:
print(processed[0])
print(len(tokenized_dataset))
print(model.config)
import os
os.environ["WANDB_DISABLED"] = "true"


{'input_ids': [51, 25, 15902, 345, 703, 284, 466, 340, 1909, 30, 685, 5188, 47, 60, 350, 25, 3878, 13, 1374, 389, 345, 30, 50256, 40, 1101, 1804, 880, 13, 6930, 329, 4737, 13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results-small",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    logging_dir="./logs-small",
    eval_strategy="epoch",  # no evaluation during training
    save_strategy="no",        # no checkpoints saved
    report_to="none"           # disable reporting to tools like WandB
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,  # use the full dataset now
    eval_dataset=tokenized_val_dataset,
)

trainer.train()


Epoch,Training Loss,Validation Loss


In [ ]:
## saving the model 
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")

In [ ]:
def generate_response(input_text):
    # Move model to GPU
    model.to('cuda')  # Ensure the model is on the GPU

    # Tokenize input text
    input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors='pt')

    # Move input_ids to GPU
    input_ids = input_ids.to('cuda')  # Move input_ids to the same device as the model

    # Generate response from the model
    output = model.generate(input_ids, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.9, temperature=0.8)

    # Decode the response
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Filter out any unwanted tokens such as [SEP] or 'P:'
    response = response.replace('[SEP]', '').replace('P:', '').strip()

    return response

# Test with a custom sentence entered by the user
user_input = "Hello, i am sick and tired "  # Example, change this input to test with different sentences
response = generate_response(user_input)
print(f"Response: {response}")


In [20]:
eval_results = trainer.evaluate(tokenized_test_dataset)

# Print evaluation results
print("Evaluation Results on Test Set:")
print(eval_results)

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Evaluation Results on Test Set:
{'eval_loss': 4.165743827819824, 'eval_runtime': 39.8335, 'eval_samples_per_second': 24.301, 'eval_steps_per_second': 6.075, 'epoch': 5.0}


In [21]:
pip install evaluate
pip install bert_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==1

In [24]:
import evaluate
bleu = evaluate.load("bleu")
bertscore = evaluate.load("bertscore")

def evaluate_model(test_dataset):
    predictions = []
    references = []

    for example in test_dataset:
        input_text = tokenizer.decode(example['input_ids'], skip_special_tokens=True)

        inputs = tokenizer(input_text + tokenizer.eos_token, return_tensors='pt', padding=True).to('cuda')

        output = model.generate(
            **inputs,
            max_new_tokens=50,
            do_sample=True,
            no_repeat_ngram_size=2,
            top_k=50,
            top_p=0.9,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        predictions.append(generated_text)

        # ✅ FIXED: decode label ids to text
        reference = tokenizer.decode(example['labels'], skip_special_tokens=True)
        references.append(reference)

    # Compute BLEU — needs tokenized references
    bleu_results = bleu.compute(
        predictions=predictions,
        references=[[ref.split()] for ref in references]
    )

    # Compute BERTScore — raw string references
    bertscore_results = bertscore.compute(
        predictions=predictions,
        references=references,
        lang='en'
    )

    print(f"BLEU Score: {bleu_results}")
    print(f"BERTScore: {bertscore_results}")

# Run evaluation
evaluate_model(tokenized_test_dataset)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BLEU Score: {'bleu': 0.002416618581689708, 'precisions': [0.2997300700054378, 0.07419847478184863, 0.036574801017921925, 2.467003823855927e-06], 'brevity_penalty': 0.3610678654582681, 'length_ratio': 0.49537092088946266, 'translation_length': 408254, 'reference_length': 824138}
BERTScore: {'precision': [1.0, 0.9627022743225098, 1.0, 0.9999999403953552, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9868755340576172, 0.9853189587593079, 0.9850596189498901, 0.993739128112793, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9040141105651855, 0.9213390946388245, 0.9333121180534363, 0.9899694323539734, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999998807907104, 0.9782976508140564, 0.9517185091972351, 0.9885143041610718, 1.0, 0.9999998807907104, 0.9893696308135986, 0.9999998807907104, 0.919927179813385, 0.9892134070396423, 0.9721818566322327, 0.9592976570129395, 1.0, 0.999999940395355

In [23]:
pip install bert_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing in

In [25]:
import evaluate

In [29]:
from IPython.display import FileLink

# Assuming model.safetensors is in the current working directory
FileLink(r'/kaggle/working/trained_model.zip')


/kaggle/working/trained_model.zip

In [28]:
import shutil
from IPython.display import FileLink

# Zip the entire directory or just specific files
shutil.make_archive('/kaggle/working/trained_model', 'zip', './')  # zips everything in current directory

# Display download link
FileLink('model_files.zip')

/kaggle/working/model_files.zip

In [ ]:
import os
import torch
import nltk
import numpy as np
from datasets import Dataset, DatasetDict, load_from_disk, Dataset as HFDataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM, # *** Changed for DialoGPT ***
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
    # Seq2Seq specific classes removed if not needed elsewhere
)
import evaluate
import logging
from tqdm.auto import tqdm

# --- Basic Setup ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nltk.download('punkt', quiet=True)

# --- Configuration ---
# *** Paths for DialoGPT and Re-ranker ***
DIALOGPT_MODEL_DIR = "./kaggle/working/trained_model"  # <-- Directory containing your fine-tuned DialoGPT files
RERANKER_MODEL_NAME = "bert-base-uncased"
RERANKER_OUTPUT_DIR = "/kaggle/working/reranker_bert_dialogpt" # <-- Adjusted output dir name
DATASET_PATH = "/kaggle/input/endsem-eval-nlp" # Path to original dataset arrow files

# Generation Params for Candidate Generation (DialoGPT)
NUM_RETURN_SEQUENCES = 5
NUM_BEAMS = 5
MAX_HISTORY_LENGTH = 450 # Max tokens for history part (adjust based on DialoGPT limits)
MAX_RESPONSE_LENGTH = 60 # Max tokens for the generated response part

# Re-ranker Training Params (Same as before)
RERANKER_MAX_LENGTH = MAX_HISTORY_LENGTH + MAX_RESPONSE_LENGTH + 20 # Ample room for combined text + SEP
RERANKER_BATCH_SIZE = 16
RERANKER_EPOCHS = 2
RERANKER_LR = 3e-5

# --- Load Datasets and DialoGPT Components ---
logging.info("Loading datasets and fine-tuned DialoGPT model/tokenizer...")
try:
    # Load original datasets
    val_dataset_orig = load_from_disk(os.path.join(DATASET_PATH, "val.arrow"))
    test_dataset_orig = load_from_disk(os.path.join(DATASET_PATH, "test.arrow"))
    raw_datasets = DatasetDict({'validation': val_dataset_orig, 'test': test_dataset_orig})

    # *** Load DialoGPT model and tokenizer ***
    dialogpt_tokenizer = AutoTokenizer.from_pretrained(DIALOGPT_MODEL_DIR)
    dialogpt_model = AutoModelForCausalLM.from_pretrained(DIALOGPT_MODEL_DIR).to(device)
    dialogpt_model.eval() # Set to evaluation mode

    # *** Set Padding for DialoGPT (GPT2-style) ***
    if dialogpt_tokenizer.pad_token is None:
        logging.warning("DialoGPT tokenizer has no pad token, setting to eos_token.")
        dialogpt_tokenizer.pad_token = dialogpt_tokenizer.eos_token
    dialogpt_tokenizer.padding_side = "left" # Important for decoder-only generation

    # Load BERTScore metric
    bertscore_metric = evaluate.load("bertscore")

except Exception as e:
    logging.error(f"Error loading models/datasets: {e}", exc_info=True)
    exit()

# --- Step 2: Generate Candidates using DialoGPT for Validation Set ---
logging.info(f"Generating {NUM_RETURN_SEQUENCES} candidates using DialoGPT for validation set...")

validation_candidates = []
validation_histories_original = [] # Store original format for re-ranker input
validation_targets = []

for example in tqdm(raw_datasets['validation'], desc="Generating Validation Candidates (DialoGPT)"):
    history_original = example['input_text'] # Keep original for re-ranker input text
    target_text = example['target_text']
    validation_histories_original.append(history_original)
    validation_targets.append(target_text)

    # Prepare input for DialoGPT: Replace [SEP] with EOS and add final EOS
    history_formatted = history_original.replace("[SEP]", dialogpt_tokenizer.eos_token) + dialogpt_tokenizer.eos_token

    inputs = dialogpt_tokenizer(
        history_formatted,
        return_tensors="pt",
        max_length=MAX_HISTORY_LENGTH, # Truncate history if too long
        truncation=True,
        padding=False # Padding handled later or by generate if needed for batching (but doing one by one here)
    ).to(device)

    input_ids = inputs['input_ids']
    current_input_length = input_ids.shape[1]

    # Calculate max_length for generate: current input + desired response length
    generate_max_length = current_input_length + MAX_RESPONSE_LENGTH

    with torch.no_grad():
        outputs = dialogpt_model.generate(
            input_ids,
            max_length=generate_max_length,
            num_beams=NUM_BEAMS,
            num_return_sequences=NUM_RETURN_SEQUENCES,
            pad_token_id=dialogpt_tokenizer.pad_token_id,
            eos_token_id=dialogpt_tokenizer.eos_token_id,
            early_stopping=True,
            # top_k=50, # Optional sampling parameters if beam search is too slow/rigid
            # top_p=0.95,
            # temperature=0.7
        )

    # *** Decode only the generated part for DialoGPT ***
    generated_sequences = outputs[:, current_input_length:]
    decoded_outputs = dialogpt_tokenizer.batch_decode(generated_sequences, skip_special_tokens=True)

    # Clean up potential artifacts if needed (e.g., dangling EOS)
    cleaned_outputs = [text.replace(dialogpt_tokenizer.eos_token, "").strip() for text in decoded_outputs]
    validation_candidates.append(cleaned_outputs)

logging.info(f"Generated DialoGPT candidates for {len(validation_candidates)} validation examples.")

# --- Step 3: Prepare Re-ranker Training Data (Identical Logic) ---
logging.info("Preparing training data for the re-ranker...")

reranker_train_data = {'text': [], 'label': []}

for i in tqdm(range(len(validation_histories_original)), desc="Scoring DialoGPT Candidates"):
    history = validation_histories_original[i] # Use original history format
    candidates = validation_candidates[i]      # Use DialoGPT candidates
    target = validation_targets[i]

    # Calculate BERTScore for each candidate against the target
    try:
        scores = bertscore_metric.compute(
            predictions=candidates,
            references=[target] * len(candidates),
            lang="en", device=device, batch_size=8 # Adjust batch size if OOM
        )
        f1_scores = scores['f1']
    except Exception as e:
        logging.warning(f"BERTScore failed for example {i}. Skipping. Error: {e}")
        f1_scores = [0.0] * len(candidates)

    # Create training instances: (history + candidate, score)
    for j, candidate in enumerate(candidates):
        # Use original history format for re-ranker input
        combined_text = history + " [SEP] " + candidate # Use [SEP] consistently here
        reranker_train_data['text'].append(combined_text)
        reranker_train_data['label'].append(f1_scores[j] if f1_scores and j < len(f1_scores) else 0.0)

reranker_dataset = HFDataset.from_dict(reranker_train_data)
logging.info(f"Re-ranker dataset created with {len(reranker_dataset)} examples (from DialoGPT candidates).")

# --- Step 4: Train Re-ranker Model (Identical Code) ---
logging.info(f"Loading and training re-ranker model: {RERANKER_MODEL_NAME}")

# Load re-ranker tokenizer and model (for regression)
reranker_tokenizer = AutoTokenizer.from_pretrained(RERANKER_MODEL_NAME)
reranker_model = AutoModelForSequenceClassification.from_pretrained(
    RERANKER_MODEL_NAME,
    num_labels=1 # Regression
).to(device)

# Tokenize the re-ranker dataset
def tokenize_reranker(examples):
    tokenized = reranker_tokenizer(
        examples['text'],
        truncation=True,
        padding="max_length",
        max_length=RERANKER_MAX_LENGTH
    )
    tokenized['labels'] = [float(label) for label in examples['label']]
    return tokenized

tokenized_reranker_dataset = reranker_dataset.map(tokenize_reranker, batched=True, num_proc=2) # Use multiprocessing
tokenized_reranker_dataset = tokenized_reranker_dataset.remove_columns(['text'])

# Data collator for the re-ranker
reranker_data_collator = DataCollatorWithPadding(tokenizer=reranker_tokenizer)

# Training arguments for the re-ranker
reranker_training_args = TrainingArguments(
    output_dir=RERANKER_OUTPUT_DIR,
    num_train_epochs=RERANKER_EPOCHS,
    per_device_train_batch_size=RERANKER_BATCH_SIZE,
    learning_rate=RERANKER_LR,
    weight_decay=0.01,
    logging_dir=f"{RERANKER_OUTPUT_DIR}/logs",
    logging_steps=100,
    save_strategy="epoch",
    save_total_limit=1,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

# Trainer for the re-ranker
reranker_trainer = Trainer(
    model=reranker_model,
    args=reranker_training_args,
    train_dataset=tokenized_reranker_dataset,
    tokenizer=reranker_tokenizer,
    data_collator=reranker_data_collator,
)

logging.info("Starting re-ranker training...")
reranker_trainer.train()
logging.info("Re-ranker training finished.")
reranker_trainer.save_model()
logging.info(f"Re-ranker model saved to {RERANKER_OUTPUT_DIR}")


# --- Step 5: Apply Re-ranking on Test Set using DialoGPT Candidates ---
logging.info("Applying re-ranking to the test set using DialoGPT candidates...")

# Load the trained re-ranker (Trainer should have loaded the best)
# Or load manually:
# reranker_tokenizer = AutoTokenizer.from_pretrained(RERANKER_OUTPUT_DIR)
# reranker_model = AutoModelForSequenceClassification.from_pretrained(RERANKER_OUTPUT_DIR).to(device)
reranker_model.eval()


final_predictions = []
ground_truths = []

for example in tqdm(raw_datasets['test'], desc="Processing Test Set (DialoGPT + Re-rank)"):
    history_original = example['input_text']
    target = example['target_text']
    ground_truths.append(target)

    # 1. Generate candidates using DialoGPT (same as validation loop)
    history_formatted = history_original.replace("[SEP]", dialogpt_tokenizer.eos_token) + dialogpt_tokenizer.eos_token
    inputs = dialogpt_tokenizer(history_formatted, return_tensors="pt", max_length=MAX_HISTORY_LENGTH, truncation=True).to(device)
    input_ids = inputs['input_ids']
    current_input_length = input_ids.shape[1]
    generate_max_length = current_input_length + MAX_RESPONSE_LENGTH

    with torch.no_grad():
        outputs = dialogpt_model.generate(
            input_ids, max_length=generate_max_length, num_beams=NUM_BEAMS,
            num_return_sequences=NUM_RETURN_SEQUENCES, pad_token_id=dialogpt_tokenizer.pad_token_id,
            eos_token_id=dialogpt_tokenizer.eos_token_id, early_stopping=True
        )
    generated_sequences = outputs[:, current_input_length:]
    candidates_decoded = dialogpt_tokenizer.batch_decode(generated_sequences, skip_special_tokens=True)
    candidates = [text.replace(dialogpt_tokenizer.eos_token, "").strip() for text in candidates_decoded]


    # 2. Prepare inputs for re-ranker
    reranker_inputs_text = [history_original + " [SEP] " + cand for cand in candidates]
    reranker_inputs_tokenized = reranker_tokenizer(
        reranker_inputs_text,
        return_tensors="pt", truncation=True, padding=True,
        max_length=RERANKER_MAX_LENGTH
    ).to(device)

    # 3. Get scores from re-ranker
    with torch.no_grad():
        reranker_outputs = reranker_model(**reranker_inputs_tokenized)
        scores = reranker_outputs.logits.squeeze(-1).cpu().numpy()

    # 4. Select best candidate
    best_candidate_index = np.argmax(scores)
    final_prediction = candidates[best_candidate_index]
    final_predictions.append(final_prediction)

logging.info("Re-ranking complete. Evaluating final DialoGPT + Re-ranked predictions...")

# --- Final Evaluation (Identical Code) ---
bleu_metric = evaluate.load("bleu")
bertscore_metric = evaluate.load("bertscore")

# Post-process for metrics
decoded_preds = [pred.strip() for pred in final_predictions]
decoded_labels = [label.strip() for label in ground_truths]
decoded_preds = [pred if pred else "<empty>" for pred in decoded_preds]
decoded_labels = [label if label else "<empty>" for label in decoded_labels]

# BLEU
tokenized_preds_for_bleu = [nltk.word_tokenize(pred.lower()) for pred in decoded_preds]
tokenized_labels_for_bleu = [[nltk.word_tokenize(label.lower())] for label in decoded_labels]
bleu_result = bleu_metric.compute(predictions=tokenized_preds_for_bleu, references=tokenized_labels_for_bleu)

# BERTScore
bertscore_result = bertscore_metric.compute(predictions=decoded_preds, references=decoded_labels, lang="en", device=device)

final_metrics = {
    "reranked_dialogpt_bleu": round(bleu_result["bleu"], 4),
    "reranked_dialogpt_bertscore_f1": round(np.mean(bertscore_result["f1"]), 4),
    "reranked_dialogpt_bertscore_precision": round(np.mean(bertscore_result["precision"]), 4),
    "reranked_dialogpt_bertscore_recall": round(np.mean(bertscore_result["recall"]), 4),
}

logging.info(f"Final Evaluation Metrics (DialoGPT + Re-ranking): {final_metrics}")

# Remember to compare this to the baseline DialoGPT performance (just using the top beam candidate without re-ranking)